# Heat - Basics and Advanced Features

In [1]:
from IPython.display import HTML, IFrame, Image

## Heat Introduction

### Heat is an Orchestration service of OpenStack

* Prepare infrastructure for your cloud application
  * Some support of actual applicaiton deployment
* Automate OpenStack API calls
* Human- and machine-readable representation
* Treat your infrastructure as code

**Whatever you can make with a series of OpenStack API calls - it can be done with Heat!**

* API calls for CRUD operations on OpenStack services entities
* calls are made in correct order when needed

## Heat architecture

In [2]:
heat_arch={'url': "https://docs.google.com/drawings/d/1e_BOmSe9L89azynhZn3G93it0sxUVqewKKyxewforDg/pub?w=640&h=480"}

In [3]:
Image(**heat_arch)

### API Services

+ **heat-api** - REST API endpoint for OpenStack API
+ **heat-api-cfn** - REST API compatible with AWS CFN API
+ **heat-api-cloudwatch** - specialized API where ``heat-cfntools`` posts in-instance metrics


### heat-engine - does most of the work

+ parses template and builds a graph of dependencies
+ generates internal resources representation
+ executes the graph calling OpenStack clients' APIs
+ horizontally scalable
  - multiple workers per engine process
  - multiple engines (on different hosts)

## Template formats

### 3 supported formats

* AWS-compatible
* AWS yaml representation
* Native Heat Orchestration Template (HOT)

### AWSTemplateFormatVersion

* Historically the first supported format
* Written in JSON
* Format is compatible with AWS CloudFormation
  * Your (simple) templates will run both on AWS CFN and OpenStack

```json
{
  "AWSTemplateFormatVersion": "2010-09-09",
  "Description": "Example AWS CFN template",
  "Parameters" : {
    "KeyName" : {
      "Type" : "String"
    }
  },
  "Resources": {
    "CfnLBUser" : {
      "Type" : "AWS::IAM::User"
    },
    "CfnLBAccessKey" : {
      "Type" : "AWS::IAM::AccessKey"
    }
  }
}
```

### HeatTemplateFormatVersion

* YAML version of AWS CFN templates
* created due to popular demand

```python
yaml.dumps(json.loads(template))
```

```yaml
HeatTemplateFormatVersion: 2012-12-12
Description: >
  Example Heat template (AWS-style)
Parameters:
  KeyName:
    Type: String
Resources:
  CfnLBUser:
    Type: AWS::IAM::User
  CfnLBAccessKey:
    Type: AWS::IAM::AccessKey
```

### Native HOT format

* written in YAML
* most of development and innovation happens here
  * not necessary to keep AWS compatibility

```yaml
heat_template_format: 2010-09-09
description: Example HOT template
parameters:
  key_name:
    type: string
resources:
  password:
    type: OS::Heat::RandomString
```

## Template anatomy

### Six top-level sections
* Template version specifier
* Resources

* Parameters (optional)
* Outputs (optional)
* Description (optional)
* Parameter groups (optional, for UI tools)

```yaml
heat_template_format: 2015-04-30
```

```yaml
parameters:
  image:
    type: string
    default: TestVM
    description: A secret image to boot
    label: Image
    hidden: True
    constraints:
    - custom_constraint: glance.image
      description: Must be a valid Glance image
 ```

```yaml
resources:
  my_server:
    type: OS::Nova::Server
    properties:
      image: { get_param: image }
      flavor: m1.small
```

```yaml
outputs:
  server_ip:
    description: Private IP of the server
    value: { get_attr: [ my_server, networks, private, 0 ] }
```

### Intrinsic functions

* `get_resource` - insert the UUID of the referenced resource
* `get_param` - insert the value of the specified parameter
* `get_attr` - insert an attribute of specified resource
  * can dig into complex attributes by key or index
* `get_file` - fetch the file from URI and insert verbatim
* `str_replace` - replaces substrings in a string with provided values
* `repeat` (*new in Kilo*) - repeat template snippets
* ... and some more


## Resources

* describe some entity in an OpenStack service
  * usually that entity has a UUID in some DB of some OS service
  * usually Heat's resource UUID is equal to the UUID of the thing it describes
* three types of resources supported
  * **AWS compatible ones** - `AWS::...::...`, API-compatible with AWS CFN
  * **Native OpenStack** - `OS::...:...`, no limits on API, most innovation here
  * **Custom resources** - can be written as `stevedore` plugins

### Resource description
* `type` - one of the available resource types
* `depends_on` - for explicit dependencies
* `metadata` - any key-value pairs
* `properties` - arguments to the API call, schema is resource specific
* `update_policy` - how to update resource, resource-specific
* `delete_policy` - `[ Delete | Retain | Snapshot ]`, resource-specific 

### Built-in service help

* `heat resource-type-list`
  * shows the list of all currently registered resource types
  * *caveat* - not all might be actually available to the user
* `heat resource-type-show <type::of::resource>`
  * shows a required structure of a resource as JSON template
  * includes description and limitations of resource's properties

## Environments

## DRY - Grouping resources

## Moar template tricks

## Deploying software with Heat